In [36]:
import psycopg2
import pandas as pd
from jinja2 import Environment, FileSystemLoader
from config import config

params = config()
conn = psycopg2.connect(**params)

zona = 'tornavacas'

capitulos = [
    ('Objetivo', 'objetivo'),
    ('Descripción de las actuaciones', 'descripcion'),
    ('Características de la Red Natura 2000 en la que se encuentran las parcelas de actuación', 'caracteristicas'),
    ('Efectos de las quemas prescritas sobre la RED NATURA 2000', 'efectos'),
    ('Afecciones a los habitats de interés comunitario presentes en el ZEC', 'afeccion_habitats'),
    ('Afecciones a los taxones de interés comunitario presentes en el ZEC', 'afeccion_taxones'),
    ('Conclusiones', 'conclusiones')
]

### HABITATS AFECTADOS

habitats_afectados = pd.read_sql('''
                SELECT distinct a.codue, a.generico, a.nom_comun, a.alianza, a.spsalianza
                FROM atlashabitats2005_extremadura a, quema_{}_todas b
                WHERE st_intersects(st_transform(b.geom, 23030), a.geom)
                '''.format(zona), conn)

display(habitats_afectados)

##  HABITATS PARCELAS

# habitats_parcelas = pd.read_sql('''
#             WITH ab as (
#                 SELECT
#                     a.nombre,
#                     codue as codigo,
#                     descripcio, 
#                     st_area(st_transform(a.geom, 25830)) / 10000 as sup_parcela,
#                     st_area(st_intersection(b.geom, st_transform(a.geom, 23030))) / 10000 as superficie,
#                     st_area(st_intersection(b.geom, st_transform(a.geom, 23030))) / st_area(st_transform(a.geom, 23030)) * 100 as porcentaje
#                 FROM atlashabitats2005_extremadura b, quema_{}_todas a
#                 WHERE st_intersects(st_transform(a.geom, 23030), b.geom)
#                 GROUP BY a.nombre, b.codue, b. descripcio, a.parcela, b.geom, a.geom
#                 ORDER BY a.parcela
#             )
#             SELECT 
#                 ab.*,
#                 Sum(St_Area(St_Intersection(atlas.geom, St_Transform(zec.geom, 23030)))) / 10000 as sup_hab_zec,
#                 ab.superficie * 100 * 10000 / Sum(St_Area(St_Intersection(atlas.geom, St_Transform(zec.geom, 23030)))) as sup_hab_zec
#             FROM ab, atlashabitats2005_extremadura atlas, sierra_gredos_y_valle_jerte zec
#             WHERE st_intersects(atlas.geom, st_transform(zec.geom, 23030)) and ab.codigo = atlas.codue and ab.descripcio = atlas.descripcio
#             GROUP BY ab.nombre, ab.codigo, ab.descripcio, ab.sup_parcela, ab.superficie, ab.porcentaje
#             '''.format(zona), conn)

# habitats_parcelas.columns = ['Parcela', 'Código UE', 'Descripción', 'Superficie Parcela', 'Superficie del hábitat en la parcela', 'Porcentaje de parcela ocupado por el hábitat', 'Superficie del hábitat en el ZEC', 'Porcentaje de la superficie del hábitat en la parcela sobre el total del hábitat en el ZEC']
display(habitats_parcelas)

env = Environment(loader=FileSystemLoader('static/afeccion'))
template =env.get_template('base.html')
output = template.render(
    capitulos = capitulos,
    habitats_afectados = habitats_afectados,
    habitats = habitats_parcelas
)

with open('static/afeccion/afeccion.html', 'w') as f:
    f.write(output)

,codue,generico,nom_comun,alianza,spsalianza
0,4090,Piornales,Piornal con escoba negra guadarrámico,Genistion floridae Rivas-Martínez 1974,None
1,5120,Piornales,Piornales bejarano-gredenses de piorno serrano,Genistion floridae Rivas-Martínez 1974,"Adenocarpus argyrophyllus, Adenocarpus hispani..."
2,5120,Piornales,Piornales serranos orosubmediterráneos bejaran...,Cytision oromediterranei Tüxen in Tüxen & Ober...,"Echinospartum barnadesii var. hirsutum, Junipe..."
3,9230,Melojares,Melojar acidófilo guadarrámico y oroibérico-so...,Quercion pyrenaicae Rivas Goday ex Rivas-Martí...,"Genista falcata, Physospermum cornubiense, Que..."


,Parcela,Código UE,Descripción,Superficie Parcela,Superficie del hábitat en la parcela,Porcentaje de parcela ocupado por el hábitat,Superficie del hábitat en el ZEC,Porcentaje de la superficie del hábitat en la parcela sobre el total del hábitat en el ZEC
0,Parcela 1,5120,Formaciones montanas de Genista purgans,0.499272,0.486953,97.530310,27016.387708,0.001802
1,Parcela 10,5120,Formaciones montanas de Genista purgans,3.446293,3.446371,100.000000,27016.387708,0.012757
2,Parcela 2,5120,Formaciones montanas de Genista purgans,0.392823,0.392832,100.000000,27016.387708,0.001454
3,Parcela 3,5120,Formaciones montanas de Genista purgans,0.662955,0.660062,99.561442,27016.387708,0.002443
4,Parcela 4,5120,Formaciones montanas de Genista purgans,0.797826,0.797844,100.000000,27016.387708,0.002953
5,Parcela 5,5120,Formaciones montanas de Genista purgans,7.901258,7.901437,100.000000,27016.387708,0.029247
6,Parcela 6,4090,Brezales oromediterráneos endémicos con aliaga,3.777422,0.146860,3.887744,26005.488803,0.000565
7,Parcela 6,5120,Formaciones montanas de Genista purgans,3.777422,3.630648,96.112256,27016.387708,0.013439
8,Parcela 6,9230,Robledales galaico-portugueses con Quercus rob...,3.777422,0.146860,3.887744,21049.584193,0.000698
9,Parcela 7,5120,Formaciones montanas de Genista purgans,0.598629,0.598642,100.000000,27016.387708,0.002216
